In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

np.set_printoptions(precision=2)

In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
print(raw_feature.shape)
label = np.genfromtxt('/Users/andrey/Dropbox/Hacking/Research/VisiProg2/analysis/thesis/visiprog/data/label.csv', delimiter = ',').astype(int)
label = label - 1

(5245, 82)


# Baseline approach
STSIM-M approach, no training

In [3]:
X_STSIMM, M = train_covariance(raw_feature)

In [4]:
np.savetxt('STSIM_M.csv', M, fmt='%.2f', delimiter=',')
accuracy = leave_one_sample_out(X_STSIMM)
print(accuracy)

0.871687321258


# Fisher linear discriminant approach

Exact solution, however trained with semantic labels ...

In [5]:
X_fisher, M_fisher = fisher_discriminant(X_STSIMM, label)

In [6]:
np.savetxt('FDA_semantic.csv', M, fmt='%.2f', delimiter=',')
accuracy = leave_one_sample_out(X_fisher)
print(accuracy)

0.990276453765


Now train using ViSiProg label

In [7]:
groups, N = readVSPLabel()


In [8]:
visiprog_label = np.zeros((N,))

count = 0
for g in groups:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if visiprog_label[i] != 0:
            label = visiprog_label[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        visiprog_label[i] = label


In [9]:
print(visiprog_label[:50])

# np.savetxt('visiprog_label.csv', visiprog_label)

[   0.  171.  171.  171.    0.    0.    0.    0.    0.  101.  101.  101.
  101.  101.    0.    0.    0.    0.  171.  171.  171.    0.    0.    0.
    0.  101.  101.  101.  101.    0.    0.    0.  171.    0.  171.  171.
    0.    0.    0.    0.  101.  101.  101.    0.    0.    0.    0.    0.
    0.    0.]


In [10]:
model = LFDA_VISIPROG()
model.fit(X_STSIMM, visiprog_label)

X_vsp = model.transform(X_STSIMM)


In [11]:
accuracy_vsp = leave_one_sample_out(X_vsp)
print(accuracy_vsp)

0.968732125834


In [12]:
5245 * (accuracy_vsp - accuracy)

-112.9999999999999